In [44]:
# instruciton datasetを今回のSFT形式に変換する

In [13]:
import os

#dataフォルダ内をリセット
os.system("rm -rf data/*")

0

In [1]:
from datasets import load_dataset
import json

/home/hatakeyama/miniconda3/envs/textprocess/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# ichikara

In [15]:
# 理研のichikara dataset (CC-NC-NDライセンス)
dataset=load_dataset("p1atdev/ichikara-instruction",'20231221-003')["train"]

In [16]:
print(len(dataset))

1899


In [17]:
"""
本番はこっち｡
以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい
"""

#question_template="### 指示：以下の質問に答えなさい。 ### 質問："
question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n"
answer_template="\n### 応答:\n"

In [19]:
n_instructions=1500
ichikara_list=[]

for n_instructions in [1500]:
    output_path=f"data/ichikara_{n_instructions}.jsonl"
    with open(output_path,"w") as f:
        f.write("")
    loader=iter(dataset)
    for i in range(n_instructions+300):
        original_record=next(loader)
        q=original_record["text"]
        a=original_record["output"]
        text=f"{question_template}{q}{answer_template}{a}"

        line=json.dumps({"text":text},ensure_ascii=False)

        eval_prefix=""
        if i>=n_instructions:
            eval_prefix=".eval"
        else:
            ichikara_list.append(line)
 
        with open(output_path+eval_prefix,"a") as f:
            f.write(line+"\n")


ichikara_list=list(set(ichikara_list))

# dolly

In [20]:
d_dataset=load_dataset("kunishou/databricks-dolly-15k-ja")["train"]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [22]:
len(d_dataset)

15015

In [23]:
#question_template2="### 指示：以下の質問に答えなさい。"
#question_template2="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい｡"
#question_template2="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 質問:\n"
dolly_list=[]
for n_instructions in [14000]:
    output_path=f"data/dolly{n_instructions}.jsonl"
    with open(output_path,"w") as f:
        f.write("")
    loader=iter(d_dataset)
    for i in range(n_instructions+1000):
        original_record=next(loader)
        if "input" in original_record:
            inp=original_record["input"]
        else:
            inp=""
        q=original_record["instruction"]
        a=original_record["output"]
        if inp=="":
            text=f"{question_template}{q}{answer_template}{a}"
        else:
            text=f"{question_template}{q}\n\n### 入力:\n{inp}\n{answer_template}{a}"
        eval_prefix=""

        line=json.dumps({"text":text},ensure_ascii=False)
        if i>=n_instructions:
            eval_prefix=".eval"
        else:
            dolly_list.append(line)
        with open(output_path+eval_prefix,"a") as f:
            f.write(line+"\n")

dolly_list=list(set(dolly_list))

# oasst

In [24]:
oasst = load_dataset("kunishou/oasst1-chat-44k-ja")["train"]
len(oasst)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

44042

In [25]:
oasst_list=[]

for n_instructions in [43000]:
    output_path=f"data/oasst{n_instructions}.jsonl"
    with open(output_path,"w") as f:
        f.write("")
    loader=iter(oasst)
    for i in range(n_instructions+1000):
        original_record=next(loader)["conversations"]
        q2=""
        a2=""
        q=original_record[0]["value"]
        a=original_record[1]["value"]

        if len(original_record)==4:
            q2=original_record[2]["value"]
            a2=original_record[3]["value"]

            text=f"{question_template}{q}{answer_template}{a} ### 指示:\n{q2}\n\n### 応答:\n{a2}"
        else:
            text=f"{question_template}{q}{answer_template}{a}"

        eval_prefix=""

        line=json.dumps({"text":text},ensure_ascii=False)
        if i>=n_instructions:
            eval_prefix=".eval"
        else:
            oasst_list.append(line)
 
        with open(output_path+eval_prefix,"a") as f:
            f.write(line+"\n")


oasst_list=list(set(oasst_list))

In [27]:
len(oasst_list),len(dolly_list),len(ichikara_list)

(39416, 13990, 1500)

In [28]:
#両方
n_ichi=1500
n_dolly=13500
n_oasst=39000
output_path=f"data/dolly_{n_dolly}_oasst_{n_oasst}_ichi{n_ichi}.jsonl"
with open(output_path,"w") as f:
    f.write("")

for i in range(n_dolly):
    with open(output_path,"a") as f:
        line=json.dumps({"text":dolly_list[i]},ensure_ascii=False)
        f.write(line+"\n")
for i in range(n_oasst):
    with open(output_path,"a") as f:
        line=json.dumps({"text":oasst_list[i]},ensure_ascii=False)
        f.write(line+"\n")
for i in range(n_ichi):
    with open(output_path,"a") as f:
        line=json.dumps({"text":ichikara_list[i]},ensure_ascii=False)
        f.write(line+"\n")




In [29]:
import glob
eval_path_list=glob.glob("data/*.eval")
eval_data=[]
for eval_path in eval_path_list:
    with open(eval_path) as f:
        line=f.readline()
        eval_data.append(line.strip())
with open(output_path+".eval","w") as f:
    for line in eval_data:
        f.write(line+"\n")

# Jaster
- コンペでは使ってはいけないので注意

In [13]:
import glob
json_path_list=glob.glob("../4_eval/llm-leaderboard/artifacts/jaster:v3/jaster/1.1.0/evaluation/train/*.json")

#test,dev
#json_path_list+=glob.glob("../4_eval/llm-leaderboard/artifacts/jaster:v3/jaster/1.1.0/evaluation/dev/*.json")
#json_path_list+=glob.glob("../4_eval/llm-leaderboard/artifacts/jaster:v3/jaster/1.1.0/evaluation/test/*.json")

In [14]:
len(json_path_list)

51

In [15]:
import random
jaster_data_list=[]

for path in json_path_list:
    with open(path,"r") as f:
        records=json.load(f)
    instruction=records["instruction"]  
    for record in records["samples"]:
        d={
            "text":instruction+"\n"+record["input"],
            "output":record["output"]
        }
        jaster_data_list.append(d)
random.shuffle(jaster_data_list)
len(jaster_data_list)

178956

In [17]:
import pandas as pd
#df=pd.DataFrame(jaster_data_list)
#df.to_csv("data/jaster.csv")

In [40]:
for n_instructions in [100,1000,2500,5000,10000,50000,140000]:
    output_path=f"data/jaster_{n_instructions}.jsonl"
    with open(output_path,"w") as f:
        f.write("")
    loader=iter(jaster_data_list)
    jaster_list=[]
    for i in range(n_instructions+300):
        original_record=next(loader)
        q=original_record["text"]
        a=original_record["output"]
        text=f"{question_template}{q}{answer_template}{a}"

        line=json.dumps({"text":text},ensure_ascii=False)

        eval_prefix=""
        if i>=n_instructions:
            eval_prefix=".eval"
        else:
            jaster_list.append(line)
 
        with open(output_path+eval_prefix,"a") as f:
            f.write(line+"\n")